# Introduction

This jupyter notebook goes through different versions of the gradient descent optimization algorithm.

The first section is a basic GD for a single variable function, where the analytical gradient of the function (the first derivate) is given.

The second section is a GD for a function with two variables, where the analytical expression for the derivate is not given. Here, a numerical function to calculate the gradient is used.

The third section is not a true gradient descent, but rather a discreet version of it. A two variable function is given and the algorithm finds the minimum in a discreet 2D-matrix.

The fourth section is similar to the second section, but generalized to work with a function of any dimension.

In [2]:
# The most import python libaries are imported for the notebook. For more information on what
# specific modules contain, please reference the appropriate documentation
import random
import sympy
import numpy as np 
import math as m
import plotly as plt
import itertools as itools
import pandas as pd

# GD for single variable, analytic

In [92]:
#The general idea of GD is as follows:
#   1) pick a random starting point of your function
#   2) calculate the slope (for more than one dimension this is called the gradient) of your function
#   3) subtract your gradient from your current point on the function to get a new point
#   4) iterate steps 2) and 3) with your new point until your step size falls under you require precision
#   or you reach a maximum number of iterations

# The starting position for the algorithm is not that important for the number of iterations. Since the
# function is smooth, points far away from the minimum result in a bigger step size for the first few steps
cur_x = random.randint(-10000,10000) 
# The learning rate of the GD is a simple constant which scales the gradient. If your learning rate is too big
# you might over shoot your minimum and have to re-iterate back and fourth. Think about a balls rolling with too much energy
# so it passes through the valley and goes up the slope again.
# If your learning rate is too small, you need more iterations to get to your result.
rate = 0.1 
# The precision can be chosen however close you want to be to your true minimum. Since GD is a approximation algorithm, you
# will most likely never get to the true minimum. But you can get a difference of less than 0.000001% with only very few
# iterations
precision = m.pow(10,-9)
previous_step_size = 1 # This is what precision is compared to. If the step size is lower than the precision, we are done.
max_iters = 10000 # This is the maximum numbers of iterations after which the loop aborts.
iters = 0 # iteration counter
# For a single variable function like here, the gradient of the function is equal to the first derivate of the function.
# For multi variable functions, the gradient is a vector containing all partial derivates for each arguement of the function.
df = lambda x: 2*(x+5) 

In [93]:
while previous_step_size > precision and iters < max_iters:
    prev_x = cur_x # Store current x value in prev_x.
    cur_x = cur_x - rate * df(prev_x) # This is step 2) and 3).
    previous_step_size = abs(cur_x - prev_x) # How much have we moved in this step.
    iters = iters+1 # iteration count
    print("Iteration",iters,"\nX value is",cur_x) # Print iterations

# Print end result plus a reasonable guess at to what the true minimum might have been.
print("The local minimum occurs at ", cur_x)
print("Probably the true minimum is at ", round(cur_x, abs(int(m.log10(precision)))-3))

Iteration 1 
X value is 4936.6
Iteration 2 
X value is 3948.28
Iteration 3 
X value is 3157.6240000000003
Iteration 4 
X value is 2525.0992
Iteration 5 
X value is 2019.0793600000002
Iteration 6 
X value is 1614.263488
Iteration 7 
X value is 1290.4107904
Iteration 8 
X value is 1031.32863232
Iteration 9 
X value is 824.062905856
Iteration 10 
X value is 658.2503246848
Iteration 11 
X value is 525.60025974784
Iteration 12 
X value is 419.48020779827203
Iteration 13 
X value is 334.5841662386176
Iteration 14 
X value is 266.6673329908941
Iteration 15 
X value is 212.3338663927153
Iteration 16 
X value is 168.86709311417223
Iteration 17 
X value is 134.0936744913378
Iteration 18 
X value is 106.27493959307023
Iteration 19 
X value is 84.01995167445618
Iteration 20 
X value is 66.21596133956494
Iteration 21 
X value is 51.97276907165195
Iteration 22 
X value is 40.57821525732156
Iteration 23 
X value is 31.46257220585725
Iteration 24 
X value is 24.1700577646858
Iteration 25 
X value is 1

In [89]:
print('We have found a local minimum at \n{} with a delta of \n{} running \n{} iterations'.format(cur_x, previous_step_size, iters))

We have found a local minimum at 
-4.999999951897902 with a delta of 
9.81675185585118e-10 running 
1147 iterations


# GD for two variables, non-analytical, continous

In [2]:
# For this section, there are two main problems:
# 1) we need two variables, so we need can't use floats for our data types
# 2) we don't have an analytical expression which we can evaluate for the gradient.

# Sets the two variable function
def f(x1, x2):
    return (x1-18)**2+(x2-2)**2

# Define individual partial differentials. Both functions return the gradient triangle of the function for one argument
# by changing only one of the arguements and not altering the other.
# The function is evaluated two times, once +0.001 and once -0.001 the current point on the axis in question. Then the slope
# is calculated by simply subtracting both values and dividing by the distance between them (0.002)
def pdiffx1(position):
    return (f(position[0].item()+10**-3, position[1].item()) - f(position[0].item()-10**-3, position[1].item()))/(2*10**-3)

def pdiffx2(position):
    return (f(position[0].item(), position[1].item()+10**-3) - f(position[0].item(), position[1].item()-10**-3))/(2*10**-3)

In [95]:
# The current position now is not a point, but rather a set of coordinates, which are here supply in form of a numpy-column-vector
# Starting position is set to a random point
cur_pos = np.matrix([random.randint(-10000, 10000),random.randint(-10000, 10000)]).transpose() 
rate = 0.1 # see first section
precision = 10**-5 # see first section
step = np.matrix([1, 1]) # see first section. Note, that our step is now also a vector, instead of a float value
max_iters = 10000 # see first section
iters = 0 # see first section

In [96]:
# Since we can't compare a vector to a scalar, we need to first calculate the norm of that vector. This represents the total "length"
# of that vector.
while np.linalg.norm(step) > precision and iters < max_iters:
    prev_pos = cur_pos # save current position
    # Since our position is the form of a vector, we can provide the gradient as a vector too and just subtract the two
    # with the inbuild numpy function for matrices. Notice, that numpy automatically returns row vectors instead of column-
    # vectors, so we have to transpose the gradient before subtracting it
    cur_pos = cur_pos - rate * np.matrix([pdiffx1(cur_pos), pdiffx2(cur_pos)]).transpose()
    # Since both positions are numpy matrices, the step is simply the difference
    step = abs(prev_pos-cur_pos)
    print("Iteration",iters,"\nX value is\n",cur_pos) #Print iterations
    iters += 1

Iteration 0 
X value is
 [[ 5082.8]
 [-7347.6]]
Iteration 1 
X value is
 [[ 4069.84]
 [-5877.68]]
Iteration 2 
X value is
 [[ 3259.472]
 [-4701.744]]
Iteration 3 
X value is
 [[ 2611.178]
 [-3760.995]]
Iteration 4 
X value is
 [[ 2092.542]
 [-3008.396]]
Iteration 5 
X value is
 [[ 1677.634]
 [-2406.317]]
Iteration 6 
X value is
 [[ 1345.707]
 [-1924.654]]
Iteration 7 
X value is
 [[ 1080.166]
 [-1539.323]]
Iteration 8 
X value is
 [[  867.732]
 [-1231.058]]
Iteration 9 
X value is
 [[ 697.786]
 [-984.447]]
Iteration 10 
X value is
 [[ 561.829]
 [-787.157]]
Iteration 11 
X value is
 [[ 453.063]
 [-629.326]]
Iteration 12 
X value is
 [[ 366.05 ]
 [-503.061]]
Iteration 13 
X value is
 [[ 296.44 ]
 [-402.049]]
Iteration 14 
X value is
 [[ 240.752]
 [-321.239]]
Iteration 15 
X value is
 [[ 196.202]
 [-256.591]]
Iteration 16 
X value is
 [[ 160.561]
 [-204.873]]
Iteration 17 
X value is
 [[ 132.049]
 [-163.498]]
Iteration 18 
X value is
 [[ 109.239]
 [-130.399]]
Iteration 19 
X value is
 [[ 

# GD for two variables, non analytical, discreet

In [97]:
def evalu(x, y):
    return (x-6)**2+(y-8)**2

In [98]:
# This section is a bit different from the others and closer to a use case problem (although not a very good one)
# The premise is, that instead of a continous function, we have a discreet result space. What this means is,
# that our function arguements can't take every real number, but only certain values. Maybe we want to determine the optiomal
# number of solar panels to install. With the first algorithm we might get 5.2335 solar panels, which is hard to install.
# So this sections tries to emulate the method behind gradient descent for a discreet scenario.

# Problems to think about:
# Since we can only access certain values for arguements, simply using the normal
# cur_pos = cur_pos - rate * gradient
# won't really be usable. Instead, think about what gradient descent does. It looks at the space surrounding your
# point and takes a step towards the direction with the steepest descent. Ideally, our algorithm should emulate
# that behaviour

# The following describes this algorithm for a 2D-result space
np.set_printoptions(precision=3) # This is simply setting formatting parameters for numpy
mat_GD = np.array([[None]*10]*10) # fill result space
x1 = round(len(mat_GD)/2) # set initial point to the center of the matrix
x2 = round(len(mat_GD)/2) # set initial point to the center of the matrix
# The surrounding of a point is simply each combination of one step back or one step forward or no steps for all 
# axis of our result space. This can be achieved with a nested loop:
# for i in [-1, 0, 1]:
#     for j in [-1, 0, 1]:
#         print((i,j))
#
# will return all relative cells around your current cell. The product function of the module itertools does exactly that.
permut = list(itools.product([-1, 0, 1], repeat=2)) # get all relative steps to access your adjacent points
finished = False # start loop control variable
current_minimum = evalu(x1, x2)
num_of_evals = 0
while not finished:
    i = 0
    """This loop will iterate over all adjacent cells in the matrix and evalutate the function at those points if it
    hasn't been evaluated before."""
    for step in permut: # For every possible step ...
        if mat_GD[x1+step[0], x2+step[1]]==None: # ... we check if we already have that value ...
            mat_GD[x1+step[0], x2+step[1]] = evalu(x1+step[0], x2+step[1]) # ... and if not, we calculate if.
            num_of_evals +=1
        else: i += 1
        if mat_GD[x1+step[0], x2+step[1]] <= current_minimum: # Then we compare to our current minimum ...
            # ... and if the minimum around us is different from our current spot, we remember the step and save the new
            # minimum.
            current_minimum = mat_GD[x1+step[0], x2+step[1]]
            new_x1 = x1+step[0]
            new_x2 = x2+step[1]
    # Here we print the current matrix for each iterations, so we can see the steps we took each iterations
    print(pd.DataFrame(mat_GD))
    # When we have looked at all the surrrounding cells, we check if our current coordinates are the same, as the coordinates
    # of the minimum. If yes, we found the local minimum, else we take a step towards the new minimum
    if x1 == new_x1 and x2 == new_x2:
        finished = True
    else:
        x1 = new_x1
        x2 = new_x2
print('finished')
print('minimum found at x1 = {}, x2 = {}, z = {} with {} evaluations compared to {}'.format(x1, x2, mat_GD[x1,x2], num_of_evals, mat_GD.size))

# This algorithm lacks two important properties:
# 1) It can only be used for 2 dimensional functions. This should be fairly simple to edit.
# 2) It lacks a proper edge detection, which is much more serious. Ideally, the algorithm would check,
# if the adjacent cells exists or are out of bounds and would not evaluate them

      0     1     2     3     4     5     6     7     8     9
0  None  None  None  None  None  None  None  None  None  None
1  None  None  None  None  None  None  None  None  None  None
2  None  None  None  None  None  None  None  None  None  None
3  None  None  None  None  None  None  None  None  None  None
4  None  None  None  None    20    13     8  None  None  None
5  None  None  None  None    17    10     5  None  None  None
6  None  None  None  None    16     9     4  None  None  None
7  None  None  None  None  None  None  None  None  None  None
8  None  None  None  None  None  None  None  None  None  None
9  None  None  None  None  None  None  None  None  None  None
      0     1     2     3     4     5     6     7     8     9
0  None  None  None  None  None  None  None  None  None  None
1  None  None  None  None  None  None  None  None  None  None
2  None  None  None  None  None  None  None  None  None  None
3  None  None  None  None  None  None  None  None  None  None
4  None 

In [ ]:
pd.DataFrame(mat_GD)

# GD for multivariable, non-analytic, continous

In [3]:
# This section is the most advanced form of the algorithm in this notebook. It is a GD, for a function
# with any number of arguements, with no analytical expression require. To use this algorithm, one only needs the definition
# of the function (can be a complex simulation) and the number of arguements one wants to look at.
# Until the sections is cleaned up into an actual module, one need only to change the function f and the number of arguements.

# Sets the function to optimize
def f(args):    
    return (args[0]-1293)**2+(args[1]-2)**2+(args[2]-3)**2+(args[3]-5)**2+(args[4]-2000)**2+(args[5]+5634)**2

# Calculates the gradient for a function f at a certain position, which is supplied as a list. The return value is also a list,
# where each value is the partial derivate of the function for the variable at the same index.
def grad(f, position):
    grad = []
    for ID in range(len(position)):
        # For each arguement, we copy the position list twice...
        delta_pos = position.copy()
        delta_neg = position.copy()
        # ... and change the arguement in question, once positive and once negative.
        delta_pos[ID] = delta_pos[ID] + 10**-9     
        delta_neg[ID] = delta_neg[ID] - 10**-9
        # Then we evaluate, calculate the difference and divide by the distance to get a slope approximation.
        grad.append((f(delta_pos)-f(delta_neg))/(2*10**-9))
    return grad

In [4]:
num_of_args = 6
# Since we don't need a visual representation for our result space, we can simply use pythons inbuild lists instead of
# numpys matrices
# Much of the rest is the same as in second section except we iterate over each arguement.
cur_pos = [random.randint(-10000, 10000)]*num_of_args
rate = 0.1
precision = 10**-6 #This tells us when to stop the algorithm
step = [1]*len(cur_pos)
max_iters = 10000 # maximum number of iterations
iters = 0 #iteration counter

while np.linalg.norm(step) > precision and iters < max_iters:
    prev_pos = cur_pos.copy()
    for ID in range(len(cur_pos)):        
        cur_pos[ID] = cur_pos[ID] - rate * grad(f, cur_pos)[ID]
        step[ID] = abs(prev_pos[ID]-cur_pos[ID])
    print("Iteration",iters,"\nX value is\n",cur_pos) #Print iterations
    iters += 1

Iteration 0 
X value is
 [7808.832733154297, 7558.493225097656, 7552.532760620117, 7555.512992858887, 7951.883880615234, 6426.004974365234]
Iteration 1 
X value is
 [6506.471244812012, 6044.535247802734, 6044.535247802734, 6044.535247802734, 6762.771217346191, 4014.9970932006836]
Iteration 2 
X value is
 [5463.389961242676, 4834.560958862305, 4834.560958862305, 4836.051074981689, 5810.587017059326, 2083.8066024780273]
Iteration 3 
X value is
 [4628.924934387207, 3867.475597381592, 3865.985481262207, 3870.4558296203613, 5048.392621994019, 539.3012447357178]
Iteration 4 
X value is
 [3962.0979709625244, 3094.850389480591, 3094.1053314208984, 3097.085563659668, 4438.190071105957, -696.0050182342529]
Iteration 5 
X value is
 [3428.263871192932, 2476.452199935913, 2475.7071418762207, 2478.6873741149902, 3950.1770420074463, -1683.9520053863525]
Iteration 6 
X value is
 [3001.345602989197, 1981.733648300171, 1980.9885902404785, 1983.968822479248, 3560.139147758484, -2473.8998131752014]
Iterat